In [19]:
from os.path import join, expanduser
import numpy as np
import xarray as xr

In [5]:
filename = expanduser(join("~", "Downloads", "EMIT_L2A_RFL_001_20240521T232728_2414216_003.nc"))

In [8]:
engine = "netcdf4"
wvl_group = "sensor_band_parameters"

In [10]:
ds = xr.open_dataset(filename, engine=engine)
loc = xr.open_dataset(filename, engine=engine, group="location")
wvl = xr.open_dataset(filename, engine=engine, group=wvl_group)

In [13]:
wvl

<xarray.Dataset> Size: 3kB
Dimensions:           (bands: 285)
Dimensions without coordinates: bands
Data variables:
    wavelengths       (bands) float32 1kB ...
    fwhm              (bands) float32 1kB ...
    good_wavelengths  (bands) float32 1kB ...

In [14]:
data_vars = {**ds.variables}
data_vars

{'reflectance': <xarray.Variable (downtrack: 1280, crosstrack: 1242, bands: 285)> Size: 2GB
 [453081600 values with dtype=float32]
 Attributes:
     long_name:  Surface Reflectance
     units:      unitless}

In [15]:
coords = {
    "downtrack": (["downtrack"], ds.downtrack.data),
    "crosstrack": (["crosstrack"], ds.crosstrack.data),
    **loc.variables,
}

In [16]:
coords

{'downtrack': (['downtrack'],
  array([   0,    1,    2, ..., 1277, 1278, 1279])),
 'crosstrack': (['crosstrack'],
  array([   0,    1,    2, ..., 1239, 1240, 1241])),
 'lon': <xarray.Variable (downtrack: 1280, crosstrack: 1242)> Size: 13MB
 [1589760 values with dtype=float64]
 Attributes:
     long_name:  Longitude (WGS-84)
     units:      degrees east,
 'lat': <xarray.Variable (downtrack: 1280, crosstrack: 1242)> Size: 13MB
 [1589760 values with dtype=float64]
 Attributes:
     long_name:  Latitude (WGS-84)
     units:      degrees north,
 'elev': <xarray.Variable (downtrack: 1280, crosstrack: 1242)> Size: 13MB
 [1589760 values with dtype=float64]
 Attributes:
     long_name:  Surface Elevation
     units:      m,
 'glt_x': <xarray.Variable (ortho_y: 1893, ortho_x: 2327)> Size: 35MB
 [4405011 values with dtype=float64]
 Attributes:
     long_name:  GLT Sample Lookup
     units:      pixel location,
 'glt_y': <xarray.Variable (ortho_y: 1893, ortho_x: 2327)> Size: 35MB
 [4405011 value

In [22]:
swath_ds = xr.Dataset(data_vars=data_vars, coords=coords, attrs=ds.attrs)
swath_ds

<xarray.Dataset> Size: 2GB
Dimensions:      (downtrack: 1280, crosstrack: 1242, bands: 285, ortho_y: 1893,
                  ortho_x: 2327)
Coordinates:
  * downtrack    (downtrack) int64 10kB 0 1 2 3 4 5 ... 1275 1276 1277 1278 1279
  * crosstrack   (crosstrack) int64 10kB 0 1 2 3 4 ... 1237 1238 1239 1240 1241
    lon          (downtrack, crosstrack) float64 13MB ...
    lat          (downtrack, crosstrack) float64 13MB ...
    elev         (downtrack, crosstrack) float64 13MB ...
    glt_x        (ortho_y, ortho_x) float64 35MB ...
    glt_y        (ortho_y, ortho_x) float64 35MB ...
Dimensions without coordinates: bands, ortho_y, ortho_x
Data variables:
    reflectance  (downtrack, crosstrack, bands) float32 2GB ...
Attributes: (12/38)
    ncei_template_version:             NCEI_NetCDF_Swath_Template_v2.0
    summary:                           The Earth Surface Mineral Dust Source ...
    keywords:                          Imaging Spectroscopy, minerals, EMIT, ...
    Conventions:                       CF-1.63
    sensor:                            EMIT (Earth Surface Mineral Dust Sourc...
    instrument:                        EMIT
    ...                                ...
    southernmost_latitude:             32.925119340889395
    spatialResolution:                 0.000542232520256367
    spatial_ref:                       GEOGCS["WGS 84",DATUM["WGS_1984",SPHER...
    geotransform:                      [-1.16103646e+02  5.42232520e-04 -0.00...
    day_night_flag:                    Day
    title:                             EMIT L2A Estimated Surface Reflectance...

In [23]:
GLT_NODATA_VALUE=0

glt_ds = np.nan_to_num(
    np.stack([swath_ds["glt_x"].data, swath_ds["glt_y"].data], axis=-1), nan=GLT_NODATA_VALUE
).astype(int)

glt_ds

array([[[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       ...,

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]]])

In [24]:
glt_ds.shape

(1893, 2327, 2)

In [26]:
reflectance = swath_ds["reflectance"]
reflectance

<xarray.DataArray 'reflectance' (downtrack: 1280, crosstrack: 1242, bands: 285)> Size: 2GB
[453081600 values with dtype=float32]
Coordinates:
  * downtrack   (downtrack) int64 10kB 0 1 2 3 4 5 ... 1275 1276 1277 1278 1279
  * crosstrack  (crosstrack) int64 10kB 0 1 2 3 4 5 ... 1237 1238 1239 1240 1241
    lon         (downtrack, crosstrack) float64 13MB ...
    lat         (downtrack, crosstrack) float64 13MB ...
    elev        (downtrack, crosstrack) float64 13MB ...
Dimensions without coordinates: bands
Attributes:
    long_name:  Surface Reflectance
    units:      unitless

In [29]:
ds_array = reflectance
glt_array = glt_ds
fill_value = -9999

# Build Output Dataset
if ds_array.ndim == 2:
    ds_array = ds_array[:, :, np.newaxis]
out_ds = np.full(
    (glt_array.shape[0], glt_array.shape[1], ds_array.shape[-1]),
    fill_value,
    dtype=np.float32,
)
valid_glt = np.all(glt_array != GLT_NODATA_VALUE, axis=-1)

# Adjust for One based Index - make a copy to prevent decrementing multiple times inside ortho_xr when applying the glt to elev
glt_array_copy = glt_array.copy()
glt_array_copy[valid_glt] -= 1
out_ds[valid_glt, :] = ds_array[
    glt_array_copy[valid_glt, 1], glt_array_copy[valid_glt, 0], :
]

MemoryError: Unable to allocate 4.91 PiB for an array with shape (2201263, 2201263, 285) and data type float32